In [1]:
from pyspark.sql import SparkSession


spark_session = SparkSession\
        .builder\
        .master("spark://192.168.1.153:7077") \
        .appName("novellarausell_pyspark")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .getOrCreate()
        
spark_context = spark_session.sparkContext

# Section B - Working with DataFrames and SQL

### B.1 Load the CSV file from HDFS, and call show() to verify the data is loaded correctly.

In [2]:
data_frame = spark_session.read\
    .option("header", "true")\
    .csv('hdfs://192.168.1.153:9000/parking-citations.csv')\
    .cache()

In [3]:
data_frame.show()

+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Latitude|Longitude|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+
|   1103341116|2015-12-21T00:00:00|      1251|    null|       null|            CA|           200304|null|HOND|        PA|   GY|     13147 WELBY WAY|01521|     1|        4000A1|   NO EVIDENCE OF REG|         50|    99999|    99999|
|   1103700150|2015-12-21T00:00:00|      1435|    null|       null|         

### B.2 Print the schema for the DataFrame.

In [4]:
data_frame.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



### B.3 Count the number of rows in the CSV file.

In [5]:
data_frame.count()

9257460

### B.4 Count the number of partitions in the underlying RDD.

In [6]:
data_frame.rdd.getNumPartitions()

10

### B.5 Drop the columns VIN, Latitude and Longitude.

In [7]:
data_frame_5 = data_frame.drop('VIN', 'Latitude', 'Longitude')

### B.6 Find the maximum fine amount. How many fines have this amount?

In [8]:
from pyspark.sql.types import FloatType
import pyspark.sql.functions as func

udf_tofloat = func.udf(lambda x: float(x) if x else None, FloatType())

In [9]:
data_frame_6 = data_frame_5.withColumn("Fine amount", udf_tofloat("Fine amount"))
maxFine = data_frame_6.agg(func.max('Fine amount')).first()[0]
maxFineCount = data_frame_6.filter(data_frame_6['Fine amount'] == maxFine).count()
print("The maximum fine amount is: {}\n".format(maxFine))
print("There are {} fines that have this amount".format(maxFineCount))

The maximum fine amount is: 505.0

There are 6 fines that have this amount


### B.7 Show the top 20 most frequent vehicle makes, and their frequencies

In [10]:
vehicle_makes = data_frame_5.groupby("Make")\
.count()\
.orderBy('count', ascending = False)

In [11]:
vehicle_makes.show()

+----+-------+
|Make|  count|
+----+-------+
|TOYT|1531949|
|HOND|1043276|
|FORD| 807498|
|NISS| 662097|
|CHEV| 631413|
| BMW| 422916|
|MERZ| 376830|
|VOLK| 316002|
|HYUN| 285286|
|DODG| 271590|
|LEXS| 263269|
| KIA| 217795|
|JEEP| 214965|
|AUDI| 179718|
|MAZD| 169811|
|OTHR| 154376|
| GMC| 132788|
|INFI| 120340|
|CHRY| 120317|
|ACUR| 111265|
+----+-------+
only showing top 20 rows



###  B.8 Create a User Defined Function to create a new column, ‘color long’, mapping the original colors to their corresponding values in the dictionary below. If there is no key matching the original color, use the original color

In [12]:
COLORS = {
'AL':'Aluminum', 'AM':'Amber', 'BG':'Beige', 'BK':'Black',
'BL':'Blue', 'BN':'Brown', 'BR':'Brown', 'BZ':'Bronze',
'CH':'Charcoal', 'DK':'Dark', 'GD':'Gold', 'GO':'Gold',
'GN':'Green', 'GY':'Gray', 'GT':'Granite', 'IV':'Ivory',
'LT':'Light', 'OL':'Olive', 'OR':'Orange', 'MR':'Maroon',
'PK':'Pink', 'RD':'Red', 'RE':'Red', 'SI':'Silver', 'SL':'Silver',
'SM':'Smoke', 'TN':'Tan', 'VT':'Violet', 'WT':'White',
'WH':'White', 'YL':'Yellow', 'YE':'Yellow', 'UN':'Unknown'
}

from pyspark.sql.types import StringType
import pyspark.sql.functions as func


udf_expandcolor = func.udf(lambda x: COLORS.get(x, x), StringType())

In [13]:
data_frame_8 = data_frame_5.withColumn('Color long', udf_expandcolor(data_frame_5.Color))

### B.9 Using this new column, what’s the most frequent colour value for Toyotas (TOYT)?

In [14]:
import pyspark.sql.functions as func

In [15]:
toyotas_colors_df = data_frame_8.filter(data_frame_8["Make"] == 'TOYT')\
.groupby('Color long')\
.count()\
.orderBy(func.desc('count'))\
.take(1)
print("The most frequent color value for Toyotas is: {}".format(toyotas_colors_df[0][0]))

The most frequent color value for Toyotas is: Gray


In [16]:
spark_session.stop()